# Žmogaus įsitraukimo darbo eiga su Microsoft Agent Framework

## 🎯 Mokymosi tikslai

Šiame užrašų knygelėje išmoksite įgyvendinti **žmogaus įsitraukimo** darbo eigas naudojant Microsoft Agent Framework `RequestInfoExecutor`. Šis galingas modelis leidžia sustabdyti AI darbo eigas, kad būtų surinkta žmogaus įvestis, padarant agentus interaktyviais ir suteikiant žmonėms galimybę kontroliuoti svarbius sprendimus.

## 🔄 Kas yra žmogaus įsitraukimas?

**Žmogaus įsitraukimas (HITL)** yra dizaino modelis, kuriame AI agentai sustabdo vykdymą, kad paprašytų žmogaus įvesties prieš tęsiant. Tai būtina:

- ✅ **Svarbiems sprendimams** - Gauti žmogaus patvirtinimą prieš atliekant svarbius veiksmus
- ✅ **Neaiškioms situacijoms** - Leisti žmonėms paaiškinti, kai AI nėra tikras
- ✅ **Vartotojo pageidavimams** - Paprašyti vartotojų pasirinkti iš kelių variantų
- ✅ **Atitikties ir saugumo** - Užtikrinti žmogaus priežiūrą reguliuojamoms operacijoms
- ✅ **Interaktyvioms patirtims** - Kurti pokalbių agentus, kurie reaguoja į vartotojo įvestį

## 🏗️ Kaip tai veikia Microsoft Agent Framework

Ši sistema siūlo tris pagrindinius komponentus HITL:

1. **`RequestInfoExecutor`** - Specialus vykdytojas, kuris sustabdo darbo eigą ir išskiria `RequestInfoEvent`
2. **`RequestInfoMessage`** - Bazinė klasė, skirta tipizuotiems užklausų duomenims, siunčiamiems žmonėms
3. **`RequestResponse`** - Susieja žmogaus atsakymus su pradinėmis užklausomis naudojant `request_id`

**Darbo eigos modelis:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Mūsų pavyzdys: Viešbučio rezervacija su vartotojo patvirtinimu

Mes papildysime sąlyginę darbo eigą pridėdami žmogaus patvirtinimą **prieš** siūlydami alternatyvias vietas:

1. Vartotojas prašo vietos (pvz., "Paryžius")
2. `availability_agent` patikrina, ar yra laisvų kambarių
3. **Jei nėra kambarių** → `confirmation_agent` klausia "Ar norėtumėte pamatyti alternatyvas?"
4. Darbo eiga **sustabdyta** naudojant `RequestInfoExecutor`
5. **Žmogus atsako** "taip" arba "ne" per konsolės įvestį
6. `decision_manager` nukreipia pagal atsakymą:
   - **Taip** → Rodyti alternatyvias vietas
   - **Ne** → Atšaukti rezervacijos užklausą
7. Rodyti galutinį rezultatą

Tai parodo, kaip suteikti vartotojams kontrolę agento pasiūlymams!

---

Pradėkime! 🚀


## 1 žingsnis: Importuokite reikalingas bibliotekas

Importuojame standartinius Agent Framework komponentus ir **žmogaus įsikišimui skirtas klases**:
- `RequestInfoExecutor` - Vykdytojas, kuris sustabdo darbo eigą, kad būtų gautas žmogaus įvestis
- `RequestInfoEvent` - Įvykis, kuris iškviečiamas, kai prašoma žmogaus įvesties
- `RequestInfoMessage` - Bazinė klasė, skirta tipizuotiems prašymo duomenims
- `RequestResponse` - Susieja žmogaus atsakymus su prašymais
- `WorkflowOutputEvent` - Įvykis, skirtas aptikti darbo eigos rezultatus


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## 2 žingsnis: Apibrėžkite Pydantic modelius struktūrizuotiems rezultatams

Šie modeliai apibrėžia **schemą**, kurią agentai grąžins. Mes išlaikome visus modelius iš sąlyginio darbo eigos ir pridedame:

**Nauja žmogaus įsitraukimui:**
- `HumanFeedbackRequest` - `RequestInfoMessage` subklasė, apibrėžianti užklausos duomenis, siunčiamus žmonėms
  - Sudaro `prompt` (klausimas, kurį reikia užduoti) ir `destination` (kontekstas apie nepasiekiamą miestą)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## 3 žingsnis: Sukurkite viešbučių rezervavimo įrankį

Tas pats įrankis iš sąlyginio darbo eigos - tikrina, ar paskirties vietoje yra laisvų kambarių.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 4 žingsnis: Apibrėžkite sąlygų funkcijas maršrutizavimui

Mums reikia **keturių sąlygų funkcijų** mūsų žmogaus įsitraukimo darbo eigai:

**Iš sąlyginės darbo eigos:**
1. `has_availability_condition` - Maršrutizuoja, kai viešbučiai YRA prieinami
2. `no_availability_condition` - Maršrutizuoja, kai viešbučiai NĖRA prieinami

**Nauja žmogaus įsitraukimui:**
3. `user_wants_alternatives_condition` - Maršrutizuoja, kai vartotojas sako „taip“ alternatyvoms
4. `user_declines_alternatives_condition` - Maršrutizuoja, kai vartotojas sako „ne“ alternatyvoms


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## 5 žingsnis: Sukurkite Decision Manager Executor

Tai yra **pagrindinė žmogaus įsitraukimo į procesą schema**! `DecisionManager` yra pritaikytas `Executor`, kuris:

1. **Priima žmogaus atsiliepimus** per `RequestResponse` objektus
2. **Apdoroja vartotojo sprendimą** (taip/ne)
3. **Nukreipia darbo eigą** siųsdamas pranešimus tinkamiems agentams

Pagrindinės savybės:
- Naudoja `@handler` dekoratorių, kad metodai būtų prieinami kaip darbo eigos žingsniai
- Priima `RequestResponse[HumanFeedbackRequest, str]`, kuriame yra tiek pirminis prašymas, tiek vartotojo atsakymas
- Grąžina paprastus "taip" arba "ne" pranešimus, kurie aktyvuoja mūsų sąlygų funkcijas


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## 6 žingsnis: Sukurkite pasirinktą rodymo vykdytoją

Tas pats rodymo vykdytojas iš sąlyginio darbo eigos - pateikia galutinius rezultatus kaip darbo eigos išvestį.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## 7 žingsnis: Įkelkite aplinkos kintamuosius

Konfigūruokite LLM klientą (GitHub Models, Azure OpenAI arba OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## 8 žingsnis: Sukurkite AI agentus ir vykdytojus

Sukuriame **šešis darbo eigos komponentus**:

**Agentai (apgaubti AgentExecutor):**
1. **availability_agent** - Tikrina viešbučių prieinamumą naudodamas įrankį
2. **confirmation_agent** - 🆕 Paruošia užklausą žmogaus patvirtinimui
3. **alternative_agent** - Pasiūlo alternatyvius miestus (kai vartotojas sako „taip“)
4. **booking_agent** - Skatina rezervaciją (kai kambariai yra prieinami)
5. **cancellation_agent** - 🆕 Tvarko atšaukimo pranešimą (kai vartotojas sako „ne“)

**Specialūs vykdytojai:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, kuris sustabdo darbo eigą, kol gaunamas žmogaus įvestis
7. **decision_manager** - 🆕 Individualus vykdytojas, nukreipiantis pagal žmogaus atsakymą (jau apibrėžtas aukščiau)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## 9 žingsnis: Sukurkite darbo eigą su žmogaus įsikišimu

Dabar sudarome darbo eigos grafiką su **sąlyginiu maršrutavimu**, įskaitant kelią su žmogaus įsikišimu:

**Darbo eigos struktūra:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Pagrindiniai ryšiai:**
- `availability_agent → confirmation_agent` (kai nėra kambarių)
- `confirmation_agent → prepare_human_request` (transformacijos tipas)
- `prepare_human_request → request_info_executor` (pauzė žmogaus įsikišimui)
- `request_info_executor → decision_manager` (visada - pateikia RequestResponse)
- `decision_manager → alternative_agent` (kai vartotojas sako "taip")
- `decision_manager → cancellation_agent` (kai vartotojas sako "ne")
- `availability_agent → booking_agent` (kai yra laisvų kambarių)
- Visi keliai baigiasi ties `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## 10 žingsnis: Vykdykite testą Nr. 1 - Miestas BE prieinamumo (Paryžius su žmogaus patvirtinimu)

Šis testas demonstruoja **pilną žmogaus įsitraukimo ciklą**:

1. Užklausa dėl viešbučio Paryžiuje
2. availability_agent tikrina → Nėra kambarių
3. confirmation_agent sukuria klausimą žmogui
4. request_info_executor **sustabdo darbo eigą** ir išskiria `RequestInfoEvent`
5. **Programėlė aptinka įvykį ir pateikia užklausą vartotojui konsolėje**
6. Vartotojas įveda "taip" arba "ne"
7. Programėlė siunčia atsakymą per `send_responses_streaming()`
8. decision_manager nukreipia pagal atsakymą
9. Pateikiamas galutinis rezultatas

**Pagrindinis modelis:**
- Naudokite `workflow.run_stream()` pirmajai iteracijai
- Naudokite `workflow.send_responses_streaming(pending_responses)` vėlesnėms iteracijoms
- Klausykitės `RequestInfoEvent`, kad aptiktumėte, kada reikalingas žmogaus įsikišimas
- Klausykitės `WorkflowOutputEvent`, kad užfiksuotumėte galutinius rezultatus


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## 11 žingsnis: Vykdykite 2 testą - Miestas SU prieinamumu (Stokholmas - Žmogaus įsikišimas nereikalingas)

Šis testas parodo **tiesioginį kelią**, kai kambariai yra prieinami:

1. Užklausa dėl viešbučio Stokholme
2. availability_agent patikrina → Kambariai prieinami ✅
3. booking_agent siūlo rezervaciją
4. display_result rodo patvirtinimą
5. **Žmogaus įsikišimas nereikalingas!**

Darbo eiga visiškai apeina kelią su žmogaus įsikišimu, kai kambariai yra prieinami.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Pagrindinės išvados ir geriausios praktikos žmogaus įsitraukimui

### ✅ Ką išmokote:

#### 1. **RequestInfoExecutor šablonas**
Žmogaus įsitraukimo šablonas Microsoft Agent Framework naudoja tris pagrindinius komponentus:
- `RequestInfoExecutor` - Sustabdo darbo eigą ir generuoja įvykius
- `RequestInfoMessage` - Bazinė klasė tipizuotiems užklausų duomenims (subklasuokite ją!)
- `RequestResponse` - Susieja žmogaus atsakymus su pradinėmis užklausomis

**Svarbus supratimas:**
- `RequestInfoExecutor` pats negauna įvesties - jis tik sustabdo darbo eigą
- Jūsų programos kodas turi klausytis `RequestInfoEvent` ir surinkti įvestį
- Turite iškviesti `send_responses_streaming()` su žemėlapiu, kuris susieja `request_id` su vartotojo atsakymu

#### 2. **Srautinio vykdymo šablonas**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Įvykių valdomos architektūros**
Klausykitės specifinių įvykių, kad galėtumėte valdyti darbo eigą:
- `RequestInfoEvent` - Reikalinga žmogaus įvestis (darbo eiga sustabdyta)
- `WorkflowOutputEvent` - Galutinis rezultatas pasiekiamas (darbo eiga baigta)
- `WorkflowStatusEvent` - Būsenos pokyčiai (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS ir kt.)

#### 4. **Individualūs vykdytojai su @handler**
`DecisionManager` parodo, kaip sukurti vykdytojus, kurie:
- Naudoja `@handler` dekoratorių, kad metodai būtų prieinami kaip darbo eigos žingsniai
- Priima tipizuotas žinutes (pvz., `RequestResponse[HumanFeedbackRequest, str]`)
- Maršrutizuoja darbo eigą, siųsdami žinutes kitiems vykdytojams
- Pasiekia kontekstą per `WorkflowContext`

#### 5. **Sąlyginis maršrutizavimas su žmogaus sprendimais**
Galite sukurti sąlygines funkcijas, kurios vertina žmogaus atsakymus:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Realūs pritaikymai:

1. **Patvirtinimo darbo eigos**
   - Gauti vadovo patvirtinimą prieš apdorojant išlaidų ataskaitas
   - Reikalauti žmogaus peržiūros prieš siunčiant automatinius el. laiškus
   - Patvirtinti didelės vertės operacijas prieš vykdymą

2. **Turinio moderavimas**
   - Žymėti abejotiną turinį žmogaus peržiūrai
   - Prašyti moderatorių priimti galutinį sprendimą dėl sudėtingų atvejų
   - Eskaluoti žmonėms, kai AI pasitikėjimas yra mažas

3. **Klientų aptarnavimas**
   - Leisti AI automatiškai tvarkyti įprastus klausimus
   - Eskaluoti sudėtingas problemas žmogaus agentams
   - Klausti kliento, ar jis nori kalbėtis su žmogumi

4. **Duomenų apdorojimas**
   - Prašyti žmonių išspręsti dviprasmiškus duomenų įrašus
   - Patvirtinti AI interpretacijas neaiškių dokumentų
   - Leisti vartotojams pasirinkti tarp kelių galimų interpretacijų

5. **Saugos kritinės sistemos**
   - Reikalauti žmogaus patvirtinimo prieš negrįžtamus veiksmus
   - Gauti patvirtinimą prieš prieigą prie jautrių duomenų
   - Patvirtinti sprendimus reguliuojamose pramonės šakose (sveikatos apsauga, finansai)

6. **Interaktyvūs agentai**
   - Kurti pokalbių robotus, kurie užduoda papildomus klausimus
   - Kurti vedlius, kurie padeda vartotojams atlikti sudėtingus procesus
   - Kurti agentus, kurie bendradarbiauja su žmonėmis žingsnis po žingsnio

### 🔄 Palyginimas: Su ir be žmogaus įsitraukimo

| Funkcija | Sąlyginė darbo eiga | Darbo eiga su žmogaus įsitraukimu |
|----------|---------------------|----------------------------------|
| **Vykdymas** | Vienas `workflow.run()` | Kilpa su `run_stream()` + `send_responses_streaming()` |
| **Vartotojo įvestis** | Nėra (visiškai automatizuota) | Interaktyvūs raginimai per `input()` arba UI |
| **Komponentai** | Agentai + vykdytojai | + RequestInfoExecutor + DecisionManager |
| **Įvykiai** | Tik AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent ir kt. |
| **Sustabdymas** | Nėra sustabdymo | Darbo eiga sustoja ties RequestInfoExecutor |
| **Žmogaus kontrolė** | Nėra žmogaus kontrolės | Žmonės priima svarbius sprendimus |
| **Naudojimo atvejis** | Automatizavimas | Bendradarbiavimas ir priežiūra |

### 🚀 Pažangūs šablonai:

#### Keli žmogaus sprendimo taškai
Galite turėti kelis `RequestInfoExecutor` mazgus toje pačioje darbo eigoje:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Laiko limitų valdymas
Įgyvendinkite laiko limitus žmogaus atsakymams:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Turtinga UI integracija
Vietoj `input()` integruokite su web UI, Slack, Teams ir kt.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Sąlyginis žmogaus įsitraukimas
Prašykite žmogaus įvesties tik specifinėse situacijose:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Geriausios praktikos:

1. **Visada subklasuokite RequestInfoMessage**
   - Užtikrina tipų saugumą ir validaciją
   - Leidžia turtingą kontekstą UI atvaizdavimui
   - Paaiškina kiekvieno užklausos tipo tikslą

2. **Naudokite aiškius raginimus**
   - Įtraukite kontekstą apie tai, ko prašote
   - Paaiškinkite kiekvieno pasirinkimo pasekmes
   - Laikykite klausimus paprastus ir aiškius

3. **Tvarkykite netikėtą įvestį**
   - Validuokite vartotojo atsakymus
   - Nustatykite numatytąsias reikšmes netinkamai įvesčiai
   - Pateikite aiškias klaidų žinutes

4. **Sekite užklausų ID**
   - Naudokite koreliaciją tarp request_id ir atsakymų
   - Nebandykite rankiniu būdu valdyti būsenos

5. **Projektuokite neblokuojančiai**
   - Neužblokuokite gijų laukdami įvesties
   - Naudokite asinchroninius šablonus
   - Palaikykite lygiagrečius darbo eigos egzempliorius

### 📚 Susijusios sąvokos:

- **Agentų tarpininkavimas** - Perimkite agentų skambučius (ankstesnė užrašų knygelė)
- **Darbo eigos būsenos valdymas** - Išsaugokite darbo eigos būseną tarp vykdymų
- **Daugelio agentų bendradarbiavimas** - Derinkite žmogaus įsitraukimą su agentų komandomis
- **Įvykių valdomos architektūros** - Kurkite reaktyvias sistemas su įvykiais

---

### 🎓 Sveikiname!

Jūs įvaldėte žmogaus įsitraukimo darbo eigas su Microsoft Agent Framework! Dabar žinote, kaip:
- ✅ Sustabdyti darbo eigas, kad surinktumėte žmogaus įvestį
- ✅ Naudoti RequestInfoExecutor ir RequestInfoMessage
- ✅ Tvarkyti srautinį vykdymą su įvykiais
- ✅ Kurti individualius vykdytojus su @handler
- ✅ Maršrutizuoti darbo eigas pagal žmogaus sprendimus
- ✅ Kurti interaktyvius AI agentus, kurie bendradarbiauja su žmonėmis

**Tai yra esminis šablonas, skirtas kurti patikimas ir valdomas AI sistemas!** 🚀



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Dėl svarbios informacijos rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar neteisingus aiškinimus, atsiradusius naudojant šį vertimą.
